In [9]:
import torch

from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [10]:
from helpers import CVFConfigDataset

In [11]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x)
        return x

In [12]:
dataset = CVFConfigDataset()
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [13]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

# # x = torch.tensor([[0], [0], [0], [0]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]
# x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

# y = torch.tensor([[3.0, 1.0]]) # ranks

# data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
# data.train_mask = torch.tensor([1 for _ in range(len(x))])

In [14]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 1., 1., 1.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.],
        [1., 0., 0., 1.]])

In [17]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)
    
    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        h = self.out(h)
        return torch.sum(h, dim=0)
    
    def fit(self, data_loader, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            for batch in data_loader:
                x = batch[0]
                y = batch[1]
                print(x, y)
                optimizer.zero_grad()
                out = self(x, adjacency)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()
            print("Loss:", loss)


In [18]:
gnn = VanillaGNN(dataset.nodes, 16, 1)
print(gnn)
gnn.fit(data_loader, epochs=200)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=4, out_features=16, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=16, out_features=16, bias=False)
  )
  (out): Linear(in_features=16, out_features=1, bias=True)
)
tensor([[0., 0., 0., 0.]], dtype=torch.float64) tensor([3.], dtype=torch.float64)


RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float

In [ ]:
# out = gnn(data.x, adjacency)
# print(out)

tensor([3.0000, 1.0000], grad_fn=<SumBackward1>)
